In [78]:
from bs4 import BeautifulSoup
import pandas as pd


In [79]:
text = ''

for j in range(1):
    with open(f'data/data_capitalization{j}.txt', 'r', encoding='utf-8') as file:
        text += file.read()


In [80]:
soup = BeautifulSoup(text, 'html.parser')
table = soup.find_all('td')


In [81]:
rank = []
name = []
code = []
market_cap = []
price = []
today = []
country = []


In [82]:
new = []
for row in table:
    a = row.text
    b = row.find('div', {'class': 'company-name'})
    c = row.find('div', {'class': 'company-code'})

    if b:
        name.append(b.text)

    if c:
        code.append(c.text)

    splitted = a.split('\n')
    new.append(splitted)

new = [element for element in new if '' not in element]


In [83]:
for i, data in enumerate(new):
    if i % 6 == 0:
        rank.append(data[0])

    elif i % 6 == 2:
        market_cap.append(data[0])

    elif i % 6 == 3:
        price.append(data[0])

    elif i % 6 == 4:
        today.append(data[0])

    elif i % 6 == 5:
        country.append(''.join(data[0].split(' ')[1:]))


In [84]:
df = pd.DataFrame([rank, name, code, market_cap, price, today, country]).transpose()


In [85]:
df.columns = ['Rank', 'Name', 'Code', 'Market_Cap', 'Price', 'Today', 'Country']


In [86]:
df.Rank = df.Rank.astype(int)
df.Price = df.Price.str.replace('$', '').str.replace(',', '').astype(float)
df.Today = df.Today.str.replace('%', '').astype(float)
df.Market_Cap = df.Market_Cap.str.replace('$', '')


In [87]:
df['multiplier'] = df.Market_Cap.str.split(' ')
df2 = pd.DataFrame(df['multiplier'].to_list(), columns=['Market_cap', 'multiplier'])
df2.multiplier = df2['multiplier'].map({'T': 1000000000000, 'B': 1000000000})
df.Market_Cap = df2.Market_cap.astype('float') * df2.multiplier
df = df.drop('multiplier', axis=1)
df

,Rank,Name,Code,Market_Cap,Price,Today,Country
0,1,NVIDIA,NVDA,4.219000e+12,173.00,0.95,USA
1,2,Microsoft,MSFT,3.803000e+12,511.70,1.20,USA
2,3,Apple,AAPL,3.136000e+12,210.02,0.07,USA
3,4,Amazon,AMZN,2.376000e+12,223.88,0.31,USA
4,5,Alphabet (Google),GOOG,2.233000e+12,184.70,0.51,USA
...,...,...,...,...,...,...,...
95,96,Charles Schwab,SCHW,1.691500e+11,93.10,2.02,USA
96,97,QUALCOMM,QCOM,1.675600e+11,152.61,0.95,USA
97,98,Arm Holdings,ARM,1.664900e+11,157.18,2.13,UK
98,99,Shopify,SHOP,1.650800e+11,126.75,5.63,Canada


In [88]:
df.to_excel('data.xlsx', index=False)
